In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
import pickle
from nltk.stem import PorterStemmer
from nltk.tokenize  import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import string
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
import os
from keras.layers import add
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, GlobalAveragePooling2D, CuDNNLSTM, BatchNormalization
from keras.models import Model
import numpy as np
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.utils import pad_sequences, to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python.keras import backend as K
import tensorflow as tf
from keras.models import load_model
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words
from nltk.translate import meteor_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
# temp = pickle.load(open('/content/drive/MyDrive/Information Retrieval Project/Insta Kaggle/User_dictionaries/user_dict/1_caption_to_ftr_vector.pkl', 'rb'))

In [4]:
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 1} )
sess = tf.compat.v1.Session(config=config) 
K.set_session(sess)

In [5]:
word_to_index, index_to_word, caption_dict, resnet_train_encoding, train_content, test_content, xception_train_ecoding = None, None, None, None, None, None, None

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/word_to_index.pkl', 'rb') as file:
  word_to_index = pickle.load(file)

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/index_to_word.pkl', 'rb') as file:
  index_to_word = pickle.load(file)

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/caption_dict.pkl', 'rb') as file:
  caption_dict = pickle.load(file)

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/resnet_train_encoding.pkl', 'rb') as file:
  resnet_train_encoding = pickle.load(file)

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/resnet_test_encoding.pkl', 'rb') as file:
  resnet_test_encoding = pickle.load(file)

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/train_cap_dict.pkl', 'rb') as file:
  train_content = pickle.load(file)

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/test_cap_dict.pkl', 'rb') as file:
  test_content = pickle.load(file)

with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/xception_train_encoding.pkl', 'rb') as file:
    xception_train_encoding = pickle.load(file)

In [6]:
cap_max_len = 0
for img, cap_list in train_content.items():
  for caption in cap_list:
    temp = len(caption.split())
    if(temp > cap_max_len):
      cap_max_len = temp

In [7]:
caption_image = {}
caption_ftr_vector = {}
tf_idf = {}
top_80_tf_idf = {}
user_dicts = '/content/drive/MyDrive/Information Retrieval Project/Insta Kaggle/User_dictionaries/user_dict'

for files in os.listdir(user_dicts):
  temp = pickle.load(open(user_dicts + '/' + files, 'rb'))
  split = files.split('_')
  
  if 'caption_to_ftr_vector' in files:
    caption_ftr_vector[split[0]] = temp
  
  elif 'caption_to_img_path_dict' in files:
    caption_image[split[0]] = temp
  
  elif 'top_80_dict' in files:
    top_80_tf_idf[split[0]] = temp

  elif 'sorted_tf_idf_dict' in files:
    tf_idf[split[0]] = temp
  
  else:
    print('wrong')

In [8]:
# pickle.dump(caption_image, open('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/caption_image.pkl', 'wb'))
# pickle.dump(caption_ftr_vector, open('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/caption_ftr_vector.pkl', 'wb'))
# pickle.dump(top_80_tf_idf, open('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/top_80_tf_idf.pkl', 'wb'))
# pickle.dump(tf_idf, open('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/tf_idf.pkl', 'wb'))

In [9]:
captions = []
users = []
vectors = []
img_paths = []

for user in caption_image.keys():
  for caption in caption_image[user].keys():
    if len(caption.split()) < 10:
      continue
    captions.append(caption)
    users.append(user)
    vectors.append(caption_ftr_vector[user][caption])
    img_paths.append(caption_image[user][caption])

In [10]:
df = pd.DataFrame({'caption': captions, 'img_vec': vectors, 'img_path': img_paths, 'user': users})

In [11]:
len(df)

840

In [12]:
df['caption'] = df['caption'].apply(lambda x: 'sos ' + x + ' eos')

In [13]:
unique_words = set()
for caption in df['caption']:
  temp = caption.split()
  unique_words.update(temp)

In [14]:
unique_words = list(unique_words)

In [15]:
caption_image = pickle.load(open('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/caption_ftrVector_dict.pkl', 'rb'))

In [16]:
caption_tokens = pickle.load(open('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/caption_tokens.pkl', 'rb'))
top_words = pickle.load(open('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/top_words.pkl', 'rb'))

In [17]:
# lemmatizer = WordNetLemmatizer()
# for caption in captions:
#   tokens = nltk.word_tokenize(caption)
#   tokens = [lemmatizer.lemmatize(word) for word in tokens]
#   is_eng = [(word in words.words()) for word in tokens]

#   if is_eng.count(True)/len(is_eng) < 0.5:
#     continue

  


In [18]:
images, captions, users = [], [], []

for user, posts in caption_image.items():
  for caption, image in caption_image[user].items():
    images.append(image)
    captions.append(caption)
    users.append(user)

In [19]:
temp_images, temp_captions, temp_users = [], [], []
lemmatizer = WordNetLemmatizer()
word_set = set(words.words())
for i in range(len(captions)):
  tokens = nltk.word_tokenize(captions[i])
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  is_eng = [(word in word_set) for word in tokens]
  print(i, end = ' ')

  if is_eng.count(True)/len(is_eng) < 0.5:
    continue

  temp_images.append(images[i])
  temp_captions.append(captions[i])
  temp_users.append(users[i])

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [20]:
print(len(temp_images), len(images))

1061 1637


In [21]:
captions = temp_captions
images = temp_images
users = temp_users

In [22]:
temp = []
for caption in captions:
  cur_caption = []
  for word in caption.split():
    if(len(word)) < 15:
      cur_caption.append(word)

  temp.append(' '.join(cur_caption))

captions = temp

In [23]:
images = np.array(images)
captions = np.array(captions)
users = np.array(users)

In [24]:
def pre_process(x):
    x = x.lower()
    x = re.sub(r'[^\w\s]', '', x)
    x = re.sub("[^a-z]+", " ", x)
    return x

In [25]:
for i in range(len(captions)):
  captions[i] = pre_process(captions[i])

In [26]:
shuffle = list(zip(images, captions, users))

In [27]:
random.shuffle(shuffle)

In [28]:
images, captions, users = zip(*shuffle)

In [29]:
images, captions, users = np.array(images), np.array(captions), np.array(users)

In [30]:
for i in range(len(captions)):
  captions[i] = 'sos ' + captions[i] + ' eos'

In [31]:
train_size = int(len(images) * 0.9)
train_images, train_captions, train_users = images[0:train_size], captions[0:train_size], users[0:train_size]
test_images, test_captions, test_users = images[train_size:], captions[train_size:], users[train_size:]

In [32]:
unique_words_scrape = set()
for caption in captions:
  temp = caption.split()
  unique_words_scrape.update(temp)

In [33]:
i = max(index_to_word.keys()) + 1
for word in unique_words:
    if word not in word_to_index:
        word_to_index[word] = i
        index_to_word[i] = word
        i += 1

In [34]:
i = max(index_to_word.keys()) + 1
for word in unique_words_scrape:
  if word not in word_to_index:
    word_to_index[word] = i
    index_to_word[i] = word
    i += 1

In [35]:
caption_VOCAB_SIZE = len(word_to_index) + 1
print(caption_VOCAB_SIZE)

13301


In [36]:
i=0
j=0
for caption in df['caption']:
  j+=1
  if len(caption.split()) < 10:
    i+=1
  if len(caption.split()) > cap_max_len:
    cap_max_len = len(caption.split())

for caption in captions:
  j += 1
  if len(caption.split()) < 10:
    i+=1
  if len(caption.split()) > cap_max_len:
    cap_max_len = len(caption.split())
  
print(cap_max_len)
print(i)
print(j)

408
29
1901


In [37]:
user_context = {}

for i in range(len(df)):
  if df.loc[i]['user'] in user_context:
    user_context[df.loc[i]['user']].extend(df.loc[i]['caption'].split())
  else:
    user_context[df.loc[i]['user']] = []
    user_context[df.loc[i]['user']].extend(df.loc[i]['caption'].split())

max_user_context = 0

for user in user_context:
    if len(set(user_context[user])) > max_user_context:
        max_user_context = len(set(user_context[user]))

In [38]:
max_user_context

637

In [39]:
user_list = list(user_context.keys())
context_list = list(user_context.values())

In [40]:
vectorizer = TfidfVectorizer()
context_sentences = [' '.join(tokens) for tokens in context_list]
user_context_tfidf = vectorizer.fit_transform(context_sentences)

In [41]:
for i in range(len(user_list)):
  user_context[user_list[i]] = user_context_tfidf[i]

In [42]:
df = df.sample(frac = 1)

In [43]:
df_numpy = df.to_numpy()

In [44]:
x_train, x_test, y_train, y_test = train_test_split(df_numpy[:, :-1], df_numpy[:, -1], test_size = 0.1)

In [45]:
# x_train, x_res, y_train, y_res = train_test_split(x_train, y_train, test_size = 0.95, stratify = y_train)

In [46]:
user_context_tfidf.shape

(84, 3498)

In [47]:
np.max(user_context_tfidf.A)

0.49551449444757323

In [48]:
index = {}
with open('/content/drive/MyDrive/Information Retrieval Project/glove.6B.50d.txt', encoding = 'utf8') as embedding_file:

  for line in embedding_file:

    temp = line.split()

    key = temp[0]
    embeddings = np.array(temp[1:], dtype = 'float')
    index[key] = embeddings

In [49]:
embedding_matrix = np.zeros(shape = (caption_VOCAB_SIZE, 50))

for word in word_to_index.keys():
  if word in index:
    embedding_matrix[word_to_index[word]] = index.get(word)

In [ ]:
class Decoder_flickr():
    def __init__(self, img_input_dim, cap_max_len, VOCAB_SIZE):

      self.img_input_dim = img_input_dim
      self.cap_max_len = cap_max_len
      self.VOCAB_SIZE = VOCAB_SIZE
      
      self.build_model()

    def build_model(self):
      img_input = Input(shape=(self.img_input_dim,))
      img_dropout = Dropout(0.2)(img_input)
      img_dense = Dense(256, activation='relu')(img_dropout)


      cap_input = Input(shape=(self.cap_max_len,))
      cap_embedding = Embedding(input_dim=caption_VOCAB_SIZE, output_dim=50, mask_zero=True)(cap_input)
      cap_dropout = Dropout(0.3)(cap_embedding)
      cap_lstm = LSTM(256)(cap_dropout)

      img_cap_add = add([img_dense, cap_lstm])
      decoder = Dense(256, activation='relu')(img_cap_add)
      outputs = Dense(caption_VOCAB_SIZE, activation='softmax')(decoder)

      self.model = Model (inputs=[img_input, cap_input], outputs=outputs)
    
    def getSummary(self):
        return self.model.summary()
    
    def set_embed_weights(self, layer_num, embedding_matrix):
        self.model.layers[layer_num].set_weights([embedding_matrix])
        self.model.layers[layer_num].trainable = False
    
    def compile(self):
        self.model.compile(loss="categorical_crossentropy", optimizer="adam")
    
    def train(self, epochs, batch_size, train_cap_dict, train_encoding, word_to_index, max_len):
        steps = 1600 // batch_size
        for i in range(epochs):        
            generator = self.data_generator(train_cap_dict, train_encoding, word_to_index, max_len, batch_size)
            self.model.fit_generator(generator, steps_per_epoch=steps)

    def data_generator (self, train_content, train_encoding, word_to_index, max_len, batch_size):
        X1, X2, y = [], [], []
        cur_batch = 0

        while True:
            for imageID, cap_list in train_content.items():
                cur_batch += 1
                image = train_encoding [imageID]

                for caption in cap_list:
                    indexes = [word_to_index[key] for key in caption.split() if key in word_to_index]

                    for i in range (1, len(indexes)):
                        x_cap_temp = indexes[0 : i]
                        x_cap_pred = indexes[i]

                        x_cap_temp = pad_sequences([x_cap_temp], maxlen=max_len, value=0, padding='post')[0]
                        x_cap_pred = to_categorical([x_cap_pred], num_classes=caption_VOCAB_SIZE)[0]

                        X1.append(image)
                        X2.append(x_cap_temp)
                        y.append(x_cap_pred)

                    if cur_batch == batch_size:
                        yield [[np.array(X1), np.array(X2)], np.array(y)]
                        
                        X1, X2, y = [], [], []
                        cur_batch = 0

#### flickr model train

In [ ]:
decoder = Decoder_flickr(2048, cap_max_len, caption_VOCAB_SIZE)

In [ ]:
decoder.getSummary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 408)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 408, 50)      665050      ['input_2[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 2048)         0           ['input_1[0][0]']                
                                                                                              

In [ ]:
# decoder.set_embed_weights(2, embedding_matrix)

In [ ]:
decoder.compile()
epochs = 30
batch_size = 16
flickr_train = decoder.train(epochs, batch_size, train_content, resnet_train_encoding, word_to_index, cap_max_len)

<ipython-input-49-4c3368add007>:41: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  self.model.fit_generator(generator, steps_per_epoch=steps)


100/100 [==============================] - 15s 155ms/step - loss: 1.9793


In [ ]:
decoder.model.save('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/saved_models/' + 'model_flick_30eps_new.h5')

#### scrape model train

In [50]:
flickr_model = load_model('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/saved_models/' + 'model_flick_30eps_new.h5')

In [ ]:
class Decoder_scrape():
    def __init__(self, img_input_dim, cap_max_len, VOCAB_SIZE ,con_VOCAB_SIZE):

      self.img_input_dim = img_input_dim
      self.cap_max_len = cap_max_len
      self.VOCAB_SIZE = VOCAB_SIZE
      self.con_VOCAB_SIZE = con_VOCAB_SIZE
      
      self.build_model()

    def build_model(self):
      img_input = Input(shape=self.img_input_dim)
      img_pooling = GlobalAveragePooling2D()(img_input)
      img_dropout = Dropout(0.2)(img_pooling)
      img_dense = Dense(256, activation='relu')(img_dropout)


      cap_input = Input(shape=(self.cap_max_len,))
      cap_embedding = Embedding(input_dim=self.VOCAB_SIZE, output_dim=2056, mask_zero=True)(cap_input)
      cap_dropout = Dropout(0.3)(cap_embedding)
      cap_dense = Dense(1024, activation = 'relu')(cap_dropout)
      cap_normal = BatchNormalization()(cap_dense)
      cap_dense2 = Dense(512, activation = 'relu')(cap_normal)
      cap_lstm = LSTM(256)(cap_dense2)

      # context_input = Input(shape = (self.con_VOCAB_SIZE, ))
      # context_embedding = Dense(2048, activation = 'relu')(context_input)
      # context_dropout = Dropout(0.3)(context_embedding)
      # context_dense = Dense(1024, activation = 'relu')(context_dropout)
      # context_normal = BatchNormalization()(context_dense)
      # context_dense2 = Dense(512, activation = 'relu')(context_normal)
      # context_dropout2 = Dropout(0.2)(context_dense2)
      # context_dense3 = Dense(256, activation = 'relu')(context_dropout2)
      # context_normal2 = BatchNormalization()(context_dense3)

      img_cap_add = add([img_dense, cap_lstm])
      decoder = Dense(256, activation='relu')(img_cap_add)
      outputs = Dense(self.VOCAB_SIZE, activation='softmax')(decoder)

      self.model = Model (inputs=[img_input, cap_input], outputs=outputs)
    
    def getSummary(self):
        return self.model.summary()
    
    def compile(self):
        self.model.compile(loss="categorical_crossentropy", optimizer="adam")
    
    def train(self, word_to_index, images, captions, users, batch_size, epochs, train_size):
        steps = train_size // batch_size
        for i in range(epochs):        
            generator = self.data_generator(word_to_index, images, captions, users, batch_size)
            self.model.fit_generator(generator, steps_per_epoch=steps)

    def data_generator (self, word_to_index, images, captions, users, batch_size):
        X1, X2, X3, y = [], [], [], []
        cur_batch = 0

        while True:
            for x in range(len(images)):
                cur_batch += 1

                image = images[x]

                
                indexes = [word_to_index[key] for key in captions[x].split() if key in word_to_index]

                for i in range (1, len(indexes)):
                    x_cap_temp = indexes[0 : i]
                    x_cap_pred = indexes[i]

                    x_cap_temp = pad_sequences([x_cap_temp], maxlen=self.cap_max_len, value=0, padding='post')[0]
                    x_cap_pred = to_categorical([x_cap_pred], num_classes=self.VOCAB_SIZE)[0]

                    X1.append(image)
                    # X2.append(user_context[users[x]].A)
                    X3.append(x_cap_temp)
                    y.append(x_cap_pred)

                if cur_batch == batch_size:
                    yield [[np.array(X1), np.array(X3)], np.array(y)]
                    
                    X1, X2, X3, y = [], [], [], []
                    cur_batch = 0

In [ ]:
scrape_decoder = Decoder_scrape((7, 7, 2048), cap_max_len, caption_VOCAB_SIZE, user_context_tfidf.shape[1])

In [ ]:
# scrape_decoder.model = flickr_model

In [ ]:
scrape_decoder.getSummary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 408)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 408, 2056)    27346856    ['input_2[0][0]']                
                                                                                                  
 dropout_1 (Dropout)            (None, 408, 2056)    0           ['embedding[0][0]']              
                                                                                                  
 input_1 (InputLayer)           [(None, 7, 7, 2048)  0           []                               
                                ]                                                             

In [ ]:
img_input = Input(shape=(7, 7, 2048))
img_pooling = GlobalAveragePooling2D()(img_input)
img_dropout = Dropout(0.2)(img_pooling)
img_dense = Dense(256, activation='relu')(img_dropout)


cap_input = Input(shape=(cap_max_len,))
cap_embedding = Embedding(input_dim=caption_VOCAB_SIZE, output_dim=50, mask_zero=True)(cap_input)
cap_dropout = Dropout(0.3)(cap_embedding)
cap_lstm = LSTM(256)(cap_dropout)

img_cap_add = add([img_dense, cap_lstm])
decoder = Dense(256, activation='relu')(img_cap_add)
outputs = Dense(caption_VOCAB_SIZE, activation='softmax')(decoder)

scrape_model = Model (inputs=[img_input, cap_input], outputs=outputs)

In [ ]:
scrape_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 7, 7, 2048)  0           []                               
                                ]                                                                 
                                                                                                  
 input_4 (InputLayer)           [(None, 408)]        0           []                               
                                                                                                  
 global_average_pooling2d_1 (Gl  (None, 2048)        0           ['input_3[0][0]']                
 obalAveragePooling2D)                                                                            
                                                                                            

In [ ]:
i, j = 2, 3
scrape_layers = scrape_model.layers
flickr_layers = flickr_model.layers
while True:
  scrape_layers[j].set_weights(flickr_layers[i].get_weights())
  i += 1
  j += 1
  if i == len(flickr_layers):
    break

In [ ]:
scrape_decoder.model = scrape_model

In [ ]:
scrape_decoder.compile()
epochs = 20
batch_size = 4
scrape_train = scrape_decoder.train(word_to_index, train_images.squeeze(axis = 1), train_captions, train_users, batch_size, epochs, len(train_images))

<ipython-input-51-4914f31793a5>:52: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  self.model.fit_generator(generator, steps_per_epoch=steps)


238/238 [==============================] - 48s 202ms/step - loss: 2.6753


In [ ]:
scrape_decoder.model.save('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/saved_models/' + 'model_flickr_scrape_new.h5')

In [51]:
scrape_model = load_model('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/saved_models/' + 'model_flickr_scrape_new.h5')

#### insta model train

In [ ]:
class Decoder():
    def __init__(self, img_input_dim, cap_max_len, VOCAB_SIZE ,con_VOCAB_SIZE):

      self.img_input_dim = img_input_dim
      self.cap_max_len = cap_max_len
      self.VOCAB_SIZE = VOCAB_SIZE
      self.con_VOCAB_SIZE = con_VOCAB_SIZE
      
      self.build_model()

    def build_model(self):
      img_input = Input(shape=(2048,))
      img_dropout = Dropout(0.2)(img_input)
      img_dense = Dense(256, activation='relu')(img_dropout)


      cap_input = Input(shape=(self.cap_max_len,))
      cap_embedding = Embedding(input_dim=self.VOCAB_SIZE, output_dim=50, mask_zero=True)(cap_input)
      cap_dropout = Dropout(0.3)(cap_embedding)
      cap_lstm = LSTM(256)(cap_dropout)

    #   context_input = Input(shape = (max_user_context, ))
    #   context_embedding = Embedding(input_dim = self.VOCAB_SIZE, output_dim = 50, mask_zero = True)(context_input)
    #   context_dropout = Dropout(0.2)(context_embedding)
    #   context_dense = Dense(256, activation = 'relu')(context_dropout)

    #   context_input = Input(shape = (self.con_VOCAB_SIZE, ))
    #   context_dense1 = Dense(1024, activation = 'relu')(context_input)
    #   context_normal = BatchNormalization()(context_dense1)
    #   context_dense2 = Dense(512, activation = 'relu')(context_normal)
    #   context_dropout2 = Dropout(0.2)(context_dense2)
    #   context_dense3 = Dense(256, activation = 'relu')(context_dropout2)
    #   context_normal2 = BatchNormalization()(context_dense3)

      img_cap_add = add([img_dense, cap_lstm])
      decoder = Dense(256, activation='relu')(img_cap_add)
      outputs = Dense(self.VOCAB_SIZE, activation='softmax')(decoder)

      self.model = Model (inputs=[img_input, cap_input], outputs=outputs)
    
    def getSummary(self):
        return self.model.summary()
    
    def compile(self):
        self.model.compile(loss="categorical_crossentropy", optimizer="adam")
    
    def train(self, word_to_index, x_train, y_train, user_context, batch_size, epochs, train_size):
        steps = train_size // batch_size
        for i in range(epochs):        
            generator = self.data_generator(word_to_index, x_train, y_train, user_context, batch_size)
            self.model.fit_generator(generator, steps_per_epoch=steps)
    
    def set_embed_weights(self, layer_num, embedding_matrix):
        self.model.layers[layer_num].set_weights([embedding_matrix])
        self.model.layers[layer_num].trainable = False

    def data_generator (self, word_to_index, x_train, y_train, user_context, batch_size):
        X1, X2, X3, y = [], [], [], []
        cur_batch = 0

        while True:
            for x in range(len(x_train)):
                cur_batch += 1

                image = x_train[x][1]

                
                indexes = [word_to_index[key] for key in x_train[x][0].split() if key in word_to_index]

                # user_words = set(user_context[y_train[x]])
                # context_index = [word_to_index[key] for key in user_words if key in word_to_index]

                for i in range (1, len(indexes)):
                    x_cap_temp = indexes[0 : i]
                    x_cap_pred = indexes[i]

                    x_cap_temp = pad_sequences([x_cap_temp], maxlen=self.cap_max_len, value=0, padding='post')[0]
                    x_cap_pred = to_categorical([x_cap_pred], num_classes=self.VOCAB_SIZE)[0]

                    X1.append(image)
                    # X2.append(pad_sequences([context_index], maxlen = max_user_context, value = 0, padding = 'post')[0])
                    X3.append(x_cap_temp)
                    y.append(x_cap_pred)

                if cur_batch == batch_size:
                    yield [[np.array(X1).squeeze(axis = 1), np.array(X3)], np.array(y)]
                    
                    X1, X2, X3, y = [], [], [], []
                    cur_batch = 0

In [ ]:
decoder_insta = Decoder(2048, cap_max_len, caption_VOCAB_SIZE, user_context_tfidf.shape[1])

In [ ]:
decoder_insta.model = decoder.model

In [ ]:
decoder_insta.getSummary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 401)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 401, 50)      298700      ['input_2[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 2048)         0           ['input_1[0][0]']                
                                                                                              

In [ ]:
decoder_insta.compile()
epochs = 20
batch_size = 16
decoder_insta.train(word_to_index, x_train, y_train, user_context, batch_size, epochs, len(x_train))

<ipython-input-46-c403d530110b>:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  self.model.fit_generator(generator, steps_per_epoch=steps)


47/47 [==============================] - 3s 58ms/step - loss: 5.9924


In [ ]:
decoder_insta.model.save('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/saved_models/' + 'model_flick_insta.h5')

In [ ]:
insta_model = load_model('/content/drive/MyDrive/Information Retrieval Project/Our final submissions/Midsem Eval/saved_models/' + 'model_flick_insta.h5')

In [67]:
def gen_caption(image, model):
  caption = 'sos'

  for i in range(50):
    indexes = [word_to_index[key] for key in caption.split() if key in word_to_index]
    indexes = pad_sequences([indexes], maxlen = cap_max_len)
    y_pred = model.predict([image, indexes], verbose = 0)
    y_pred = np.argmax(y_pred)
    word_pred = index_to_word[y_pred]
    caption = caption + ' ' + word_pred
    if word_pred == 'eos':
      break

  return caption

#### flickr testing

In [53]:
with open('/content/drive/MyDrive/Information Retrieval Project/flickr dataset/test_cap_dict.pkl', 'rb') as file:
  test_cap_dict = pickle.load(file)

bleu_score = 0
for imageID, caption_list in test_cap_dict.items():
  image = resnet_test_encoding[imageID].reshape((1, 2048))
  caption = gen_caption(image, flickr_model)
  bleu_score += sentence_bleu(caption_list, caption)

bleu_score /= len(test_cap_dict)

In [54]:
bleu_score

0.5077187506304797

In [61]:
met_score = 0
for imageID, caption_list in test_cap_dict.items():
  image = resnet_test_encoding[imageID].reshape((1, 2048))
  caption = gen_caption(image, flickr_model)
  caption_tokens = [nltk.word_tokenize(cap) for cap in caption_list]
  pred_tokens = nltk.word_tokenize(caption)
  met_score += meteor_score.meteor_score(caption_tokens, pred_tokens)

met_score /= len(test_cap_dict)

In [63]:
met_score

0.33956251288431155

#### scraper testing

In [70]:
bleu_score = 0
met_score = 0
for i in range(50):
  print(i, end = ' ')
  caption = gen_caption(test_images[i], scrape_model)
  bleu_score += sentence_bleu(test_captions[i], caption)
  caption_tokens = [nltk.word_tokenize(test_captions[i])]
  pred_tokens = nltk.word_tokenize(caption)
  met_score += meteor_score.meteor_score(caption_tokens, pred_tokens)

met_score /= len(test_cap_dict)
bleu_score /= len(test_cap_dict)

0 

/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

In [75]:
bleu_score *= len(test_cap_dict)
bleu_score /= 50
bleu_score

9.95330149975552e-230

In [76]:
met_score *= len(test_cap_dict)
met_score /= 50
met_score

0.10397295389549685